In [1]:
import pandas as pd
import numpy as np

import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold,GroupKFold
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm import tqdm
import gc
import re
from sklearn.metrics import roc_auc_score
import os
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import polars as pl
from pathlib import Path
from glob import glob
import json
import joblib
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
os.makedirs('abstract')

In [ ]:
import os
for i in range(106):
    os.system(f'python3 chatglm_abstract.py {i}')

0


Loading checkpoint shards: 100%|██████████| 7/7 [00:04<00:00,  1.56it/s]
Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.
100%|██████████| 2957/2957 [01:25<00:00, 34.62it/s]


0 3000
1


Loading checkpoint shards: 100%|██████████| 7/7 [00:03<00:00,  1.96it/s]
Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.
0it [00:00, ?it/s]


3000 6000
2


Loading checkpoint shards:  71%|███████▏  | 5/7 [00:02<00:00,  2.07it/s]
Traceback (most recent call last):
  File "chatglm_abstract.py", line 83, in <module>
    model = AutoModel.from_pretrained("THUDM/chatglm3-6b",trust_remote_code=True)
  File "/root/miniconda3/lib/python3.8/site-packages/transformers/models/auto/auto_factory.py", line 556, in from_pretrained
    return model_class.from_pretrained(
  File "/root/miniconda3/lib/python3.8/site-packages/transformers/modeling_utils.py", line 3502, in from_pretrained
    ) = cls._load_pretrained_model(
  File "/root/miniconda3/lib/python3.8/site-packages/transformers/modeling_utils.py", line 3948, in _load_pretrained_model
    del state_dict
KeyboardInterrupt


3


Loading checkpoint shards:  86%|████████▌ | 6/7 [00:02<00:00,  2.19it/s]
Traceback (most recent call last):
  File "chatglm_abstract.py", line 83, in <module>
    model = AutoModel.from_pretrained("THUDM/chatglm3-6b",trust_remote_code=True)
  File "/root/miniconda3/lib/python3.8/site-packages/transformers/models/auto/auto_factory.py", line 556, in from_pretrained
    return model_class.from_pretrained(
  File "/root/miniconda3/lib/python3.8/site-packages/transformers/modeling_utils.py", line 3502, in from_pretrained
    ) = cls._load_pretrained_model(
  File "/root/miniconda3/lib/python3.8/site-packages/transformers/modeling_utils.py", line 3949, in _load_pretrained_model
    gc.collect()
KeyboardInterrupt


In [2]:
t_dict = dict()
for i in tqdm(range(106)):
    a = joblib.load(f'abstract/t_dict_abstract_{i}.pkl')
    t_dict.update(a)

  2%|▏         | 2/106 [00:00<00:07, 14.01it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'abstract/t_dict_abstract_2.pkl'

In [3]:
!rm -rf abastract

In [5]:
joblib.dump(t_dict,'t_dict_abstract_all.pkl')

['t_dict_abstract_all.pkl']

In [6]:

train = pd.read_feather('data/train.feather')
valid = pd.read_feather('data/valid.feather')
test = pd.read_feather('data/test.feather')

piddf = joblib.load('data/pid_df.pkl')

data = pd.concat([train,valid,test]).reset_index(drop = True)
piddf = piddf[piddf['id'].isin(set(data['PID']))]

In [7]:

def cos_similarity(target, embedding):
    numerator = np.sum(target * embedding, axis=1)
    denominator = np.sqrt(np.sum(np.square(target)) * np.sum(np.square(embedding),axis=1))
    return numerator / denominator

ans = []
for autherID in tqdm(data['autherID'].unique()):
    pid_list = data[data['autherID'] == autherID]['PID'].to_list()
    for f1 in pid_list:
        x1 = t_dict.get(f1)
        x2 = [f for f in pid_list if f != f1]
        x2 = [t_dict.get(f) for f in x2]
        x1 = np.array(x1 , dtype = np.float32)
        x2 = np.array(x2 , dtype = np.float32)

        ans.append(list(cos_similarity(x1, x2)))

    
data['chatglm_pid_abstract_sim'] = ans

for f in ['chatglm_pid_abstract_sim']:
    data[f + '_mean'] = data[f].apply(lambda x:np.mean(x))
    data[f + '_max'] = data[f].apply(lambda x:np.max(x))
    data[f + '_min'] = data[f].apply(lambda x:np.min(x))
    data[f + '_std'] = data[f].apply(lambda x:np.std(x))
    data[f + '_median'] = data[f].apply(lambda x:np.median(x))
    
data = data.drop(['chatglm_pid_abstract_sim'],axis = 1)

100%|██████████| 15/15 [00:35<00:00,  2.37s/it]


In [8]:
data[['PID',  'autherID',  'chatglm_pid_abstract_sim_mean', 'chatglm_pid_abstract_sim_max',
       'chatglm_pid_abstract_sim_min', 'chatglm_pid_abstract_sim_std',
       'chatglm_pid_abstract_sim_median']].to_feather('chatglm_pid_abstract_sim.pkl')

In [9]:
t_dict = joblib.load(f't_dict_abstract_all.pkl')
abstract_emb = pd.DataFrame(t_dict.values()).values
svd_tmp = TruncatedSVD(n_components=16, n_iter=20, random_state=2024)
abstract_emb_svd = svd_tmp.fit_transform(abstract_emb)
abstract_emb_svd = pd.DataFrame(abstract_emb_svd,columns = [f'chatglm_abstract_emb_svd_{i}' for i in range(16)])
abstract_emb_svd = pd.concat([pd.DataFrame(t_dict.keys(),columns = ['PID']),abstract_emb_svd],axis = 1)
data = data.merge(abstract_emb_svd,on = ['PID'],how = 'left')

In [12]:
data[['PID', 'autherID', 'chatglm_abstract_emb_svd_0',
       'chatglm_abstract_emb_svd_1', 'chatglm_abstract_emb_svd_2',
       'chatglm_abstract_emb_svd_3', 'chatglm_abstract_emb_svd_4',
       'chatglm_abstract_emb_svd_5', 'chatglm_abstract_emb_svd_6',
       'chatglm_abstract_emb_svd_7', 'chatglm_abstract_emb_svd_8',
       'chatglm_abstract_emb_svd_9', 'chatglm_abstract_emb_svd_10',
       'chatglm_abstract_emb_svd_11', 'chatglm_abstract_emb_svd_12',
       'chatglm_abstract_emb_svd_13', 'chatglm_abstract_emb_svd_14',
       'chatglm_abstract_emb_svd_15']].to_feather('chatglm_pid_abstract_svd.pkl')

In [2]:
os.makedirs('title')

FileExistsError: [Errno 17] File exists: 'title'

In [3]:
import os
for i in range(106):
    os.system(f'python3 chatglm_title.py {i}')

0


Loading checkpoint shards: 100%|██████████| 7/7 [00:02<00:00,  2.58it/s]
Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.
100%|██████████| 2957/2957 [00:55<00:00, 53.37it/s]


0 3000


In [4]:
t_dict = dict()
for i in tqdm(range(106)):
    a = joblib.load(f'title/t_dict_title_{i}.pkl')
    t_dict.update(a)

100%|██████████| 1/1 [00:00<00:00,  5.40it/s]


In [5]:
!rm -rf title

In [6]:
joblib.dump(t_dict,'t_dict_title_all.pkl')

['t_dict_title_all.pkl']

In [7]:

train = pd.read_feather('data/train.feather')
valid = pd.read_feather('data/valid.feather')
test = pd.read_feather('data/test.feather')

piddf = joblib.load('data/pid_df.pkl')

data = pd.concat([train,valid,test]).reset_index(drop = True)
piddf = piddf[piddf['id'].isin(set(data['PID']))]

In [8]:

def cos_similarity(target, embedding):
    numerator = np.sum(target * embedding, axis=1)
    denominator = np.sqrt(np.sum(np.square(target)) * np.sum(np.square(embedding),axis=1))
    return numerator / denominator

ans = []
for autherID in tqdm(data['autherID'].unique()):
    pid_list = data[data['autherID'] == autherID]['PID'].to_list()
    for f1 in pid_list:
        x1 = t_dict.get(f1)
        x2 = [f for f in pid_list if f != f1]
        x2 = [t_dict.get(f) for f in x2]
        x1 = np.array(x1 , dtype = np.float32)
        x2 = np.array(x2 , dtype = np.float32)

        ans.append(list(cos_similarity(x1, x2)))

    
data['chatglm_pid_title_sim'] = ans

for f in ['chatglm_pid_title_sim']:
    data[f + '_mean'] = data[f].apply(lambda x:np.mean(x))
    data[f + '_max'] = data[f].apply(lambda x:np.max(x))
    data[f + '_min'] = data[f].apply(lambda x:np.min(x))
    data[f + '_std'] = data[f].apply(lambda x:np.std(x))
    data[f + '_median'] = data[f].apply(lambda x:np.median(x))
    
data = data.drop(['chatglm_pid_title_sim'],axis = 1)

100%|██████████| 15/15 [00:38<00:00,  2.58s/it]


In [9]:
data[['PID',  'autherID',  'chatglm_pid_title_sim_mean', 'chatglm_pid_title_sim_max',
       'chatglm_pid_title_sim_min', 'chatglm_pid_title_sim_std',
       'chatglm_pid_title_sim_median']].to_feather('chatglm_pid_title_sim.pkl')

In [10]:
t_dict = joblib.load(f't_dict_title_all.pkl')
abstract_emb = pd.DataFrame(t_dict.values()).values
svd_tmp = TruncatedSVD(n_components=16, n_iter=20, random_state=2024)
abstract_emb_svd = svd_tmp.fit_transform(abstract_emb)
abstract_emb_svd = pd.DataFrame(abstract_emb_svd,columns = [f'chatglm_title_emb_svd_{i}' for i in range(16)])
abstract_emb_svd = pd.concat([pd.DataFrame(t_dict.keys(),columns = ['PID']),abstract_emb_svd],axis = 1)
data = data.merge(abstract_emb_svd,on = ['PID'],how = 'left')

In [11]:
data.columns

Index(['PID', 'label', 'autherID', 'autherName', 'chatglm_pid_title_sim_mean',
       'chatglm_pid_title_sim_max', 'chatglm_pid_title_sim_min',
       'chatglm_pid_title_sim_std', 'chatglm_pid_title_sim_median',
       'chatglm_title_emb_svd_0', 'chatglm_title_emb_svd_1',
       'chatglm_title_emb_svd_2', 'chatglm_title_emb_svd_3',
       'chatglm_title_emb_svd_4', 'chatglm_title_emb_svd_5',
       'chatglm_title_emb_svd_6', 'chatglm_title_emb_svd_7',
       'chatglm_title_emb_svd_8', 'chatglm_title_emb_svd_9',
       'chatglm_title_emb_svd_10', 'chatglm_title_emb_svd_11',
       'chatglm_title_emb_svd_12', 'chatglm_title_emb_svd_13',
       'chatglm_title_emb_svd_14', 'chatglm_title_emb_svd_15'],
      dtype='object')

In [12]:
data[['PID', 'autherID','chatglm_title_emb_svd_0', 'chatglm_title_emb_svd_1',
       'chatglm_title_emb_svd_2', 'chatglm_title_emb_svd_3',
       'chatglm_title_emb_svd_4', 'chatglm_title_emb_svd_5',
       'chatglm_title_emb_svd_6', 'chatglm_title_emb_svd_7',
       'chatglm_title_emb_svd_8', 'chatglm_title_emb_svd_9',
       'chatglm_title_emb_svd_10', 'chatglm_title_emb_svd_11',
       'chatglm_title_emb_svd_12', 'chatglm_title_emb_svd_13',
       'chatglm_title_emb_svd_14', 'chatglm_title_emb_svd_15']].to_feather('chatglm_pid_title_svd.pkl')